In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importación de la data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('/content/drive/MyDrive/Nuclio/Capstone/Copia de BTCUSDT_velas_15.csv', sep=',',index_col=0)
df.set_index('Close time', inplace=True)
del df['Open time']
df


,Open,High,Low,Close,Volume,Quote assert volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
Close time,,,,,,,,,,
2019-01-01 00:14:59.999,3701.23,3703.72,3689.88,3697.31,190.974095,7.059148e+05,1325,106.065467,3.921042e+05,0
2019-01-01 00:29:59.999,3697.91,3698.75,3690.56,3694.20,131.742721,4.866613e+05,1073,74.943180,2.768597e+05,0
2019-01-01 00:44:59.999,3693.96,3705.00,3693.00,3702.39,171.877714,6.358759e+05,1589,95.601285,3.536763e+05,0
2019-01-01 00:59:59.999,3702.05,3713.00,3695.02,3700.31,191.772890,7.106172e+05,1547,94.245382,3.493222e+05,0
2019-01-01 01:14:59.999,3700.20,3702.73,3691.22,3701.03,139.872706,5.171780e+05,1357,74.994622,2.773194e+05,0
...,...,...,...,...,...,...,...,...,...,...
2022-12-19 23:14:59.999,16416.82,16459.18,16373.43,16382.34,3127.652070,5.132545e+07,69586,1523.437960,2.500099e+07,0
2022-12-19 23:29:59.999,16382.34,16428.99,16373.14,16423.25,2542.894310,4.169301e+07,55397,1340.155230,2.197322e+07,0
2022-12-19 23:44:59.999,16423.25,16442.35,16410.25,16440.90,2062.223700,3.387183e+07,49406,1081.777110,1.776870e+07,0


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso

from xgboost import XGBRegressor
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
resultados = []
for i in range(1442):
  
      df['LAG_CLOSE'] = df['Close'].shift(-1)

      df_T = df.iloc[-1]
      df.drop(index=df.index[-1],inplace=True, axis=0)

      def Benchmarck(df):

        l = []
        variable = ['Close', 'High', 'Low', 'Open', 'LAG_CLOSE']
        for i in df:
          if i not in variable:
            l.append(i)

        df.drop(l, axis=1, inplace=True)

      Benchmarck(df)

      target = 'LAG_CLOSE'
      features = [i for i in df.columns if i != target]

      y = df[target]
      X = df[features]

      X_train, X_test, y_train, y_test = train_test_split(
                                              X, 
                                              y, 
                                              test_size = 0.3, # tamaño del split aleatorio 
                                              # aunque el split sea aleatorio, stratify garantiza que el ratio de exitos sea el mismo en train y test
                                              random_state = 2 # Guarda el split en el seed 42 por si volvemos a ejecutar la celda, sea el mismo split
                                          )


      transform_pipe = ColumnTransformer(transformers = [
          ("scaler", MinMaxScaler(), [0,1,2,3]),
      
          
        ])

      pipe = Pipeline(steps = [
          ("transform", transform_pipe),
          
        ])


      X_train = pipe.fit_transform(X_train)
      X_test = pipe.transform(X_test)
    
      def RMSE(model,X,y):
        return round(np.sqrt(np.mean((y-model.predict(X))**2)),2)
        
      def cross_val(model,X,y):
        Kfold_rmse = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')
        return round(-Kfold_rmse.mean(),2)

      R = XGBRegressor(objective ='reg:squarederror')
      model = R.fit(X_train, y_train) 

      df_T = pd.DataFrame(df_T)
      df_T = df_T.T

      Benchmarck(df_T)

      target = 'LAG_CLOSE'
      features = [i for i in df.columns if i != target]

      y_pred = df_T[target]
      X_pred = df_T[features]

      X_pred = pipe.transform(X_pred)
      y_pred = model.predict(X_pred)

      df_T['LAG_CLOSE'] = y_pred
      df = df.append(df_T)

      lag_close = df['LAG_CLOSE'] 
      print('----------------------------------')
      print(f'Intento: {i}')
      print(f'LAG_CLOSE: {y_pred}')


      # Modelo LAG_HIGH

 
      df['LAG_HIGH'] = df['High'].shift(-1)

      df_T = df.iloc[-1]
      df.drop(index=df.index[-1],inplace=True, axis=0)

      def Benchmarck(df):

        l = []
        variable = ['Close', 'High', 'Low', 'Open', 'LAG_HIGH']
        for i in df:
          if i not in variable:
            l.append(i)

        df.drop(l, axis=1, inplace=True)

      Benchmarck(df)

      target = 'LAG_HIGH'
      features = [i for i in df.columns if i != target]

      y = df[target]
      X = df[features]

      X_train, X_test, y_train, y_test = train_test_split(
                                              X, 
                                              y, 
                                              test_size = 0.3, # tamaño del split aleatorio 
                                              # aunque el split sea aleatorio, stratify garantiza que el ratio de exitos sea el mismo en train y test
                                              random_state = 2 # Guarda el split en el seed 42 por si volvemos a ejecutar la celda, sea el mismo split
                                          )

      transform_pipe = ColumnTransformer(transformers = [
          ("scaler", MinMaxScaler(), [0,1,2,3]),
      
          
        ])

      pipe = Pipeline(steps = [
          ("transform", transform_pipe),
          
        ])


      X_train = pipe.fit_transform(X_train)
      X_test = pipe.transform(X_test)
  
      R = XGBRegressor(objective ='reg:squarederror')
      model = R.fit(X_train, y_train) 

      df_T = pd.DataFrame(df_T)
      df_T = df_T.T

      Benchmarck(df_T)

      target = 'LAG_HIGH'
      features = [i for i in df.columns if i != target]

      y_pred = df_T[target]
      X_pred = df_T[features]

      X_pred = pipe.transform(X_pred)

      y_pred = model.predict(X_pred)

      df_T['LAG_HIGH'] = y_pred
      df = df.append(df_T)

      lag_high = df['LAG_HIGH'] 
      print(f'LAG_HIGH: {y_pred}')


      # Modelo LAG_LOW

      df['LAG_LOW'] = df['Low'].shift(-1)

      df_T = df.iloc[-1]
      df.drop(index=df.index[-1],inplace=True, axis=0)

      def Benchmarck(df):

        l = []
        variable = ['Close', 'High', 'Low', 'Open', 'LAG_LOW']
        for i in df:
          if i not in variable:
            l.append(i)

        df.drop(l, axis=1, inplace=True)

      Benchmarck(df)

      target = 'LAG_LOW'
      features = [i for i in df.columns if i != target]

      y = df[target]
      X = df[features]

      X_train, X_test, y_train, y_test = train_test_split(
                                              X, 
                                              y, 
                                              test_size = 0.3, # tamaño del split aleatorio 
                                              # aunque el split sea aleatorio, stratify garantiza que el ratio de exitos sea el mismo en train y test
                                              random_state = 2 # Guarda el split en el seed 42 por si volvemos a ejecutar la celda, sea el mismo split
                                          )

      transform_pipe = ColumnTransformer(transformers = [
          ("scaler", MinMaxScaler(), [0,1,2,3]),
      
          
        ])

      pipe = Pipeline(steps = [
          ("transform", transform_pipe),
          
        ])


      X_train = pipe.fit_transform(X_train)
      X_test = pipe.transform(X_test)

      R = XGBRegressor(objective ='reg:squarederror')
      model = R.fit(X_train, y_train) 

      ## 7 Predict LAG_LOW

      df_T = pd.DataFrame(df_T)
      df_T = df_T.T

      Benchmarck(df_T)

      target = 'LAG_LOW'
      features = [i for i in df.columns if i != target]

      y_pred = df_T[target]
      X_pred = df_T[features]

      X_pred = pipe.transform(X_pred)

      y_pred = model.predict(X_pred)

      df_T['LAG_LOW'] = y_pred
      df = df.append(df_T)

      lag_low = df['LAG_LOW'] 
      print(f'LAG_LOW: {y_pred}')


      # DataFrame



      df = df.join([lag_close, lag_high])
      nueva_prediccion = df[['LAG_LOW','LAG_CLOSE','LAG_HIGH']].tail(1)
      nueva_prediccion.columns= ['Low','Close','High']
      nueva_prediccion['Open'] = df['Close'].iloc[-1]


      df = df.append(nueva_prediccion)
      df.reset_index(inplace=True)
      del df['index']
      if i == 1441:
        resultados.append(df)
      else:
        continue

Se han truncado las últimas 5000 líneas del flujo de salida.
----------------------------------
Intento: 67
LAG_CLOSE: [16625.422]
LAG_HIGH: [16662.473]
LAG_LOW: [16654.994]
----------------------------------
Intento: 68
LAG_CLOSE: [16659.123]
LAG_HIGH: [16658.17]
LAG_LOW: [16527.404]
----------------------------------
Intento: 69
LAG_CLOSE: [16624.213]
LAG_HIGH: [16627.438]
LAG_LOW: [16565.361]
----------------------------------
Intento: 70
LAG_CLOSE: [16637.717]
LAG_HIGH: [16655.576]
LAG_LOW: [16566.568]
----------------------------------
Intento: 71
LAG_CLOSE: [16633.049]
LAG_HIGH: [16660.895]
LAG_LOW: [16605.773]
----------------------------------
Intento: 72
LAG_CLOSE: [16637.877]
LAG_HIGH: [16666.07]
LAG_LOW: [16617.408]
----------------------------------
Intento: 73
LAG_CLOSE: [16656.635]
LAG_HIGH: [16662.19]
LAG_LOW: [16593.]
----------------------------------
Intento: 74
LAG_CLOSE: [16644.158]
LAG_HIGH: [16661.932]
LAG_LOW: [16483.15]
----------------------------------
Intento

<ipython-input-10-01248a4f9ada>:240: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nueva_prediccion['Open'] = df['Close'].iloc[-1]


----------------------------------
Intento: 1067
LAG_CLOSE: [17574.709]
LAG_HIGH: [17628.498]
LAG_LOW: [17436.656]
----------------------------------
Intento: 1068
LAG_CLOSE: [17564.38]
LAG_HIGH: [17644.805]
LAG_LOW: [17442.145]
----------------------------------
Intento: 1069
LAG_CLOSE: [17556.027]
LAG_HIGH: [17627.982]
LAG_LOW: [17455.195]
----------------------------------
Intento: 1070
LAG_CLOSE: [17573.082]
LAG_HIGH: [17634.227]
LAG_LOW: [17478.457]
----------------------------------
Intento: 1071
LAG_CLOSE: [17558.568]
LAG_HIGH: [17630.395]
LAG_LOW: [17465.139]
----------------------------------
Intento: 1072
LAG_CLOSE: [17558.678]
LAG_HIGH: [17634.68]
LAG_LOW: [17468.314]
----------------------------------
Intento: 1073
LAG_CLOSE: [17568.129]
LAG_HIGH: [17635.219]
LAG_LOW: [17448.78]
----------------------------------
Intento: 1074
LAG_CLOSE: [17564.094]
LAG_HIGH: [17636.438]
LAG_LOW: [17446.506]
----------------------------------
Intento: 1075
LAG_CLOSE: [17571.807]
LAG_HIGH: [

In [ ]:
df_resultados = resultados[-1]

In [ ]:
df_resultados.tail(32)
df_resultados['diff'] = df_resultados['Close'].shift(-1) - df_resultados['Close'] 
df_resultados.loc[df_resultados['diff'] > 0,'Up_DownP'] = 1
df_resultados.loc[df_resultados['diff'] < 0,'Up_DownP'] = 0
df_resultados.drop(['diff'], inplace=True, axis=1)
df_resultados

,Open,High,Low,Close,LAG_LOW,LAG_CLOSE,LAG_HIGH,Up_DownP
0,3701.230000,3703.720000,3689.880000,3697.310000,3690.560000,3694.200000,3698.750000,0.0
1,3697.910000,3698.750000,3690.560000,3694.200000,3693.000000,3702.390000,3705.000000,1.0
2,3693.960000,3705.000000,3693.000000,3702.390000,3695.020000,3700.310000,3713.000000,0.0
3,3702.050000,3713.000000,3695.020000,3700.310000,3691.220000,3701.030000,3702.730000,1.0
4,3700.200000,3702.730000,3691.220000,3701.030000,3695.460000,3699.250000,3702.390000,0.0
...,...,...,...,...,...,...,...,...
140350,17566.755859,17619.230469,17453.578125,17565.769531,17436.710938,17557.808594,17624.572266,0.0
140351,17565.769531,17624.572266,17436.710938,17557.808594,17453.591797,17551.328125,17626.128906,0.0
140352,17557.808594,17626.128906,17453.591797,17551.328125,17455.638672,17562.750000,17628.707031,1.0
140353,17551.328125,17628.707031,17455.638672,17562.750000,17453.128906,17561.871094,17625.988281,0.0


In [ ]:
df_resultados.to_csv('/content/drive/MyDrive/Nuclio/Capstone/BTC_15min_.csv')